In [185]:
import pandas as pd
import numpy as np
import math
import warnings

warnings.filterwarnings("error")

ratings = pd.read_csv("dataset/ratings.csv").drop('timestamp', axis = 1) #open csv file and drop 'timestamp' column, it is not necessary
movies = pd.read_csv("dataset/movies.csv").drop(['title', 'genres'], axis = 1) #open csv file and drop 'timestamp'  and 'genres' columns, they are not necessary
rows_count = len(ratings) #number of rows
print(f'ratings.csv has {rows_count} rows. ') #check number of rows


ratings.csv has 100836 rows. 


In [163]:
print(ratings.loc[1].to_string()) #get row 0
print(ratings.loc[0].to_string()) #get row 0

#rows_by_id = ratings.userId.value_counts() #number of rows group by id

mean_by_id = ratings.groupby('userId')['rating'].mean() #mean of ratings of every user
print(mean_by_id.loc[1]) #mean of ratings of a pecific id 
#print(mean_by_id.to_string())

#ratings.merge(ratings, left_on='userId', right_on='movieId')
user1_ratings = ratings[ratings['userId'] == 1] #df with every rows of a certain userId
user2_ratings = ratings[ratings['userId'] == 85] 
#print(user1_ratings.to_string())
#print(user2_ratings.to_string())

merge_ratings = pd.merge(user1_ratings, user2_ratings, on = 'movieId', how = 'inner') #merged df with movies of two users
print(merge_ratings.to_string())
print(merge_ratings['rating_x'].to_string())

df = ratings.groupby('userId')
print(df.groups.keys())
#users = pd.concat(ratings['userId'].groupby('userId')).drop_duplicates(keep=False) #get movies not rated by user


#movies = pd.read_csv("dataset/movies.csv").drop(['title', 'genres'], axis = 1) #open csv file and drop 'timestamp'  and 'genres' columns, they are not necessary
#print(movies.to_string())
#mvs = user1_ratings['movieId'] films already watched by the user
#print(mvs.to_string())
#print(pd.concat([movies,mvs]).drop_duplicates(keep=False).to_string()) #get movies not rated by user

#print(rows_by_id.to_string())

print(user1_ratings[user1_ratings['movieId'] == 1]['rating'])

#mean_user1 = user1_ratings['rating'].mean()
#mean(ratings_user1)

userId     1.0
movieId    3.0
rating     4.0
userId     1.0
movieId    1.0
rating     4.0
4.366379310344827
   userId_x  movieId  rating_x  userId_y  rating_y
0         1      527       5.0        85       5.0
0    5.0
dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176

In [186]:
def pearson_correlation(df_user1, df_user2): #Pearson correlation function for computing similarities between users
    merged_ratings = df_user1.merge(df_user2, on = "movieId", how = "inner") #merged df with movies rated by both users
    if merged_ratings.empty == True:
        return math.nan
    
    ratings_user1 = merged_ratings['rating_x']
    ratings_user2 = merged_ratings['rating_y']
    mean_user1 = ratings_user1.mean() #mean of ratings of user1
    mean_user2 = ratings_user2.mean() #mean of ratings of user2
    
    num = np.sum((ratings_user1 - mean_user1)*(ratings_user2 - mean_user2))
    den = np.sqrt(np.sum((ratings_user1 - mean_user1)**2)) * np.sqrt(np.sum((ratings_user2 - mean_user2)**2))
    try:
        coef = num/den
    except (RuntimeWarning, ZeroDivisionError):
        return math.nan
    
    return coef

#pearson_correlation(user1_ratings, user2_ratings)

In [215]:
def prediction(userId, movieId, ratings): #prediction function
    df_userA = ratings[ratings['userId'] == userId]
    userA_mean = df_userA['rating'].mean()
    users_for_film = ratings[ratings['movieId'] == movieId].drop(['movieId', 'rating'], axis = 1) #users that rated a specific film
    num = 0
    den = 0
    for user in users_for_film['userId']:
        df_userB = ratings[ratings['userId'] == user]
        sim = pearson_correlation(df_userA, df_userB)
        if not math.isnan(sim):
            num += sim * (df_userB[df_userB['movieId'] == movieId].iloc[0]['rating'] - df_userB['rating'].mean())
            den += sim

    try:
        div = num/den
        pred = userA_mean + div
    except (RuntimeWarning, ZeroDivisionError):
        return math.nan
    
    return pred


In [217]:
USER_A = 1
USER_A_ratings = ratings[ratings["userId"] == USER_A]
USER_A_mean = USER_A_ratings['rating'].mean()
USER_A_films = USER_A_ratings['movieId']

def func(x, dic):
    user_b_ratings = ratings[ratings['userId'] == x]
    corr = pearson_correlation(USER_A_ratings, user_b_ratings)
    if not math.isnan(corr):
        dic.update({x : corr})

def func2(x, dic):
    prd = prediction(USER_A, x, ratings)
    if not math.isnan(prd):
        dic.update({x : prd})


def main():
    #ratings = pd.read_csv("dataset/ratings.csv").drop('timestamp', axis = 1) #open csv file and drop 'timestamp' column, it is not necessary
    users = ratings.groupby('userId').groups.keys() #all the users of the dataset
    sim = dict() #dictionary of the result
    pd.DataFrame.from_dict(users).map(lambda x: func(x, sim) if(x != USER_A) else None) #call the function of similarity on every user
    top10usr = dict(sorted(sim.items(), key=lambda x:x[1], reverse=True)[:10]) #get top 10 users similar to user in input
    print(f'top 10 users similar to user in input are {top10usr}')
    
    mvs = pd.concat([movies,USER_A_films]).drop_duplicates(keep=False) #get movies not rated by user
    pred = dict()
    mvs['movieId'].map(lambda x: func2(x, pred))
    top10mvs = dict(sorted(pred.items(), key=lambda x:x[1], reverse=True)[:10]) #get top 10 users similar to user in input
    print(f'top 10 films to suggest to user in input are {top10mvs}')


main()

top 10 users similar to user in input are {146: 1.0000000000000002, 550: 1.0000000000000002, 106: 1.0, 333: 1.0, 598: 0.9999999999999998, 473: 0.9622504486493763, 511: 0.9258200997725514, 9: 0.9185586535436917, 13: 0.8783100656536798, 366: 0.8728715609439694}
3.95205140029612
2.7579640298295494
3.53985574964856
3.997729694868435
-25.483842086615198
1.9904138631856236
4.056752174914407
4.411453263070307
3.7507812274875407
4.183776580676442
5.984592311718257
13.172394938627985
4.99491674751093
4.836132423304434
3.9047937597484825
4.004296641137812
4.9369687521339065
4.450931023924653
3.7668896738050086
3.873683343899713
4.103818827527988
4.686402617442664
4.310612397491878
5.5433867978990685
4.940543458445343
4.972792470794258
3.2824588141679363
4.365015500835097
4.852306846691714
4.46153115945636
4.627098006611533
3.772100279339419
3.945182285229502
4.891900169416982
4.810525939954326
3.6065673388522046
4.216190236495658
3.192130672574492
4.370482251081448
3.0022466576317814
3.896994566